In [24]:
import re
import os
from time import gmtime, strftime
from datetime import datetime, timedelta
import unicodedata

# Importing libraries you need to install
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import requests
import bs4 as bs
from lxml import html
from tqdm import tqdm

## Now we have all the raw_text store in the txt files. And the path for the files are like:
'./Scrape_data/Scrape_data/10_k/[cik]/rawtext/[cik]_[date]'
## Our goal is to find Item7 and Item 7As in 10K and Item2 and Item3 in 10Q

In [25]:
#read the ticker library of all the tikers into ticker_library
ticker_library = pd.read_csv('tickers.csv')

#read the sp500 components into ticker_selected, 'name' is the company name and ticker is company's ticker
ticker_selected = pd.read_csv('SP500_component_stocks.csv',header = None)
ticker_selected.columns = ['name','ticker']

#display a sample of ticker_selected
display(ticker_selected)

/opt/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,name,ticker
0,Agilent Technologies Inc.,A
1,American Airlines Group Inc.,AAL
2,Advance Auto Parts Inc.,AAP
3,Apple Inc.,AAPL
4,AbbVie Inc.,ABBV
...,...,...
500,Xylem Inc.,XYL
501,Yum! Brands Inc.,YUM
502,Zimmer Biomet Holdings Inc.,ZBH
503,Zions Bancorporation N.A.,ZION


In [26]:
#build a ticker_cik_df dataframe to store ticker and its cik number
ticker_cik_df = pd.DataFrame()

#store all the tickers in a ticker_list
ticker_list = ticker_selected.ticker.unique()

#build a list cik_list for cik
cik_list = []

for ticker in ticker_list:    
    try:
        #for a given ticker, find its cik number through th ticker library
        cik_list.append(list(ticker_library[ticker_library.ticker == ticker].secfilings)[0][-10:])
        
    except:
        #if could not find cik, give it a empty cik
        cik_list.append('')

#write cik_list and ticker_list to the dataframe ticker_cik_df
ticker_cik_df['cik'] = cik_list
ticker_cik_df['ticker'] = ticker_list

#delete the tickers with empty cik number
ticker_cik_df = ticker_cik_df[ticker_cik_df['cik'] != '']

#display a sample of ticker_cik_df
ticker_cik_df

,cik,ticker
0,0001090872,A
1,0000006201,AAL
2,0001158449,AAP
3,0000320193,AAPL
4,0001551152,ABBV
...,...,...
500,0001524472,XYL
501,0001041061,YUM
502,0001136869,ZBH
503,0000109380,ZION


In [27]:
listtickers = ['AMZN','BBY','BKNG','MCD','EBAY','F','HD','TGT','WHR']
listrows = []
for j in listtickers:
    row = ticker_cik_df[ticker_cik_df['ticker'] == j]
    listrows.append(row)

In [28]:
ticker_cik_sample = pd.DataFrame()
for i in range(len(listrows)):
    ticker_cik_sample = pd.concat([ticker_cik_sample,listrows[i]],ignore_index=False)
ticker_cik_sample

,cik,ticker
39,0001018724,AMZN
64,0000764478,BBY
71,0001075531,BKNG
304,0000063908,MCD
156,0001065088,EBAY
178,0000037996,F
222,0000354950,HD
435,0000027419,TGT
485,0000106640,WHR


In [29]:
# df_cik=pd.read_csv('cik.csv',dtype=str)

In [30]:
# #posible sectors
# sectors=['COMM','DISCRETIONARY','ENERGY','FINANCE','HEALTH','INDUST','IT','MATERIALS','RE','STAPLES','UTILITY']
# #sectors we want to take a look at:
# # - Finance ('FINANCE')
# # - Health ('HEALTH')
# # - Communications (includes Twitter) ('COMM')
# # - Technology ('IT')

In [31]:
# def read_ticker(sector):
#     path='Ticker_'+str(sector)+'.csv'
#     df=pd.read_csv(path)
#     result=df.set_index('Company').join(df_cik.set_index('ticker'))
#     result=result.reset_index()
#     result=result.rename(columns={"index": "ticker"})
#     result=result[result.columns[::-1]]   
#     return result

In [32]:
# ticker_cik_finance = read_ticker('FINANCE').drop_duplicates()
# ticker_cik_health = read_ticker('HEALTH').drop_duplicates()
# ticker_cik_com = read_ticker('COMM').drop_duplicates()
# ticker_cik_it = read_ticker('IT').drop_duplicates()

In [33]:
# #Replace this name ticker_cik_sample
# ticker_cik_sample = ticker_cik_health.sample(n=10,replace=False,random_state=4)
ticker_cik_sample = ['0000049826','0000816284']

In [34]:
def try_section(data):
    management_str_list = list(combs['FIRST ITEM'].dropna())
    financial_str_list = list(combs['SECOND ITEM'].dropna())
    for i in range(len(management_str_list)):
        management_str_list[i] = management_str_list[i].upper()
        management_str = management_str_list[i]
        management_str = re.sub('\s+',' ',management_str)
        last_management = data.rfind(management_str)
        if (last_management == -1):
            continue
        else:
            break
    for i in range(len(financial_str_list)):
        financial_str_list[i] = financial_str_list[i].upper()
        financial_str = financial_str_list[i]
        financial_str = re.sub('\s+',' ',financial_str)
        if data.count(financial_str) == 0:
            financial_str = "FINANCIAL STATEMENTS AND SUPPLEMENTAL DATA"
        last_financial = data.rfind(financial_str)
        if (last_financial == -1):
            continue
        else:
            break
    return last_management, last_financial

In [35]:
#write a function to parse the raw text to gra item7 and item 7a informations:

#set input as the path of the txt file
def grab_md_10k(text_path):
    
    #open the text file and read file as upper string
    with open(text_path, encoding = "utf8") as f:
        data = f.read().upper()

    #normalize all the blanks
    data = re.sub('\s+', ' ', data)
    management_str = ["ITEM 7. MANAGEMENT'S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS", 
                      "Discussion and Analysis of Financial Condition and Results of Operations".upper(),
                      "Analysis of Financial Condition and Results of Operations".upper(),
                      "Discussion and Analysis of Financial Condition".upper()]
    financial_str = ["ITEM 8. FINANCIAL STATEMENTS AND SUPPLEMENTARY DATA", 
                     "FINANCIAL STATEMENTS AND SUPPLEMENTARY DATA", 
                     "FINANCIAL STATEMENTS AND SUPPLEMENTAL DATA"]
    for j in financial_str:
        last_financial = data.rfind(j)
        if (last_financial == -1):
            continue
        else:
            break
    for i in management_str:
        last_management = data.rfind(i)
        if (last_management == -1):
            continue
        else:
            break
    
    
    '''
    after deleting references, there should be 1 or 2 titles just above the information we need. One is in the catalog, the other 
    is in the context.
    '''
    
    #if last management string happens after the financial statement, then we cannot grab information
    if last_management > last_financial:
        
        #then we should return none 
        return None

    #else we can get a dialogue of management discussion
    else:
        return data[last_management:last_financial]    


In [36]:
#write a function to parse the raw text to gra item2 and item 3 informations:

#set input as the path of the txt file
def grab_md_10q(text_path):
    
    #open the text file and read file as upper string
    with open(text_path, encoding = "utf8") as f:
        data = f.read().upper()

    data = re.sub('\s+', ' ', data)
    management_str = ["ITEM 2. MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS", 
                      "Discussion and Analysis of Financial Condition and Results of Operations".upper(),
                      "Analysis of Financial Condition and Results of Operations".upper(),
                      "Discussion and Analysis of Financial Condition".upper(), 
                      "Analysis of Financial Condition".upper(),
                      "2. Management's Discussion and Analysis".upper(),
                      'Discussion and Analysis'.upper(),
                      "Results of Operations".upper()]
    financial_str = ["ITEM 4. CONTROLS AND PROCEDURES", 
                     "Controls and Procedures".upper(), 
                     "and Procedures".upper(), 
                     "Controls and".upper(),
                     "Controls".upper(),
                     "Procedures".upper()]
    for j in financial_str:
        last_financial = data.rfind(j)
        if (last_financial == -1):
            continue
        else:
            break
    for i in management_str:
        last_management = data.rfind(i)
        if (last_management == -1):
            continue
        else:
            break

    
    '''
    after deleting references, there should be 1 or 2 titles just above the information we need. One is in the catalog, the other 
    is in the context.
    '''

    
    #if last management string happens after the financial statement, then we cannot grab information
    if last_management > last_financial:
        
        #then we should return none 
        return None

    #else we can get a dialogue of management discussion
    else:
        return data[last_management:last_financial]    


In [37]:
#iterate through the ciks to grab 10K information
for cik in ticker_cik_sample:
    
    #given the path for 10K and 10Q
    pathname_10k = './Scrape_data/10_k/'
    pathname_10q = './Scrape_data/10_q/'
    
    #file name of 
    file_list = os.listdir(pathname_10k + cik + '/rawtext/')
    #iterate through file name to grab managemnent discussions
    for file_name in file_list:
        
        #complete a path for the text file
        text_path = pathname_10k + cik + '/rawtext/' + file_name
        #grab the management discussion part and convert it to a string
        data = grab_md_10k(text_path)
        
        #decide whether we have data to write into file
        if data != None:
        
            #move to the directory for grabbed_text 
            try:
                os.chdir(pathname_10k + cik + '/grabbed_text/')
            
            #if there isn't a directory, we compose one    
            except Exception:
                os.makedirs(pathname_10k + cik + '/grabbed_text/')
                os.chdir(pathname_10k + cik + '/grabbed_text/')
        
            #open a txt file to write data into
            file = open(file_name, 'a', encoding = "utf-8")
            
            #write data into file    
            file.write(data)
        
            #close file and return to the main directory
            file.close()
            os.chdir('..')
            os.chdir('..')
            os.chdir('..')
            os.chdir('..')

In [38]:
#iterate through the ciks to grab 10Q information
for cik in ticker_cik_sample:
    
    #given the path for 10K and 10Q
    pathname_10k = './Scrape_data/10_k/'
    pathname_10q = './Scrape_data/10_q/'
    
    #file name of 
    file_list = os.listdir(pathname_10q + cik + '/rawtext/')
    
    #iterate through file name to grab managemnent discussions
    for file_name in file_list:
        
        #complete a path for the text file
        text_path = pathname_10q + cik + '/rawtext/' + file_name
        
        #grab the management discussion part and convert it to a string
        data = grab_md_10q(text_path)
        
        #decide whether we have data to write into file
        if data != None:
        
            #move to the directory for grabbed_text 
            try:
                os.chdir(pathname_10q + cik + '/grabbed_text/')
            
            #if there isn't a directory, we compose one    
            except Exception:
                os.makedirs(pathname_10q + cik + '/grabbed_text/')
                os.chdir(pathname_10q + cik + '/grabbed_text/')
        
            #open a txt file to write data into
            file = open(file_name, 'a', encoding = "utf-8")
            
            #write data into file    
            file.write(data)
        
            #close file and return to the main directory
            file.close()
            os.chdir('..')
            os.chdir('..')
            os.chdir('..')
            os.chdir('..')
